# 0. 필요 라이브러리 및 변수, 함수 정의

## import library & package

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import random

from tqdm import tqdm
from itertools import combinations
from transformers import AutoTokenizer, BertForSequenceClassification
# from transformers import AutoModel, AutoModelForSequenceClassification
# from datasets import load_dataset, load_metric
from rank_bm25 import BM25Okapi
from sklearn.model_selection import train_test_split

**BM25: 키워드 기반 랭킹 알고리즘**
- 주어진 쿼리에 대해 문서와의 연관성을 평가하는 랭킹 함수
- Bag-of-words 개념을 사용하여 쿼리에 있는 용어가 각각의 문서에 얼마나 자주 등장하는지를 평가
    - 이때 IDF값을 곱해서 자주 등장하지 않는 단어에 더 큰 가중치를 줌.

## Define variable data

In [2]:
code_folder = "data/code/"
problem_folders = os.listdir(code_folder)

## Define function

In [3]:
# PREPROCESSING FOR CODE SCRIPT
def preprocess_script(script):
    with open(script,'r',encoding='utf-8') as file:
        lines = file.readlines()
        preprocess_lines = []
        for line in lines:
            if line.lstrip().startswith('#'): # 주석으로 시작되는 행 skip
                continue
            line = line.rstrip()
            if '#' in line:
                line = line[:line.index('#')] # 주석 전까지 코드만 저장
            line = line.replace('\n','') # 개행 문자를 모두 삭제함
            line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환
            
            if line == '': # 전처리 후 빈 라인은 skip
                continue
            
            preprocess_lines.append(line)
        preprocessed_script = '\n'.join(preprocess_lines) # 개행 문자로 합침
        
    return preprocessed_script

## Preprocessing

In [16]:
preprocess_scripts = []
problem_nums = []

# 300개 Sample code에 대한 전처리
for problem_folder in tqdm(problem_folders):
    scripts = os.listdir(os.path.join(code_folder, problem_folder))
    problem_num = problem_folder
    for script in scripts:
        script_file = os.path.join(code_folder,problem_folder,script)
        preprocessed_script = preprocess_script(script_file)

        preprocess_scripts.append(preprocessed_script)
    # 번호 목록을 만들어서 전처리한 dataframe에 함께 넣어줌
    problem_nums.extend([problem_num]*len(scripts))

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:07<00:00, 41.98it/s]


In [33]:
df = pd.DataFrame(data= {'code':preprocess_scripts, 'problem_num':problem_nums})
df.head(10)

,code,problem_num
0,"M = 9\nN = 9\ndef main():\n\tfor i in range(1,...",problem001
1,"list1 = range(1,10)\nlist2 = list1\nfor i in l...",problem001
2,"for x in range(1, 10):\n\tfor y in range(1, 10...",problem001
3,"for i in range(1,10):\n\tfor n in range(1, 10)...",problem001
4,"for i in range(1,10):\n\tfor k in range(1,10):...",problem001
5,"for i in range(1, 10):\n\tfor j in range(1, 10...",problem001
6,"for i in range(0,9):\n\tfor j in range(0,9):\n...",problem001
7,"A=range(1,10)\nfor i in A:\n x=i\n for j in ...",problem001
8,i=1\nwhile i<=9:\n\tj=1\n\twhile j<=9:\n\t\tpr...,problem001
9,"for i in range(1, 10):\n\tfor j in range(1, 10...",problem001


In [34]:
df.describe()

,code,problem_num
count,45101,45101
unique,45074,300
top,n = int(input())\nprint((n-1)//2),problem262
freq,2,153


## Create Data Set

### Tokenizer 수행, microsoft에서 개발한 사전 학습 모델인 graphcodebert 사용

In [ ]:
# # AutoTokenizer로 graphcodebert 사용하도록 설정
# tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
# # tokenizer.truncation_side = 'left'

# df['tokens'] = df['code'].apply(tokenizer.tokenize) # Sample code를 Tokenization해서 tokens 컬럼에 추가
# df['len'] = df['tokens'].apply(len) # tokens의 길이를 len 컬럼에 추가

Token indices sequence length is longer than the specified maximum sequence length for this model (773 > 512). Running this sequence through the model will result in indexing errors


In [35]:
# AutoTokenizer로 graphcodebert 사용하도록 설정
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
tokenizer.truncation_side = 'left'

tokens = []
for code in df['code']:
    tokens.append(tokenizer.tokenize(code, max_length=512, truncation=True))

df['tokens'] = tokens # Sample code를 Tokenization해서 tokens 컬럼에 추가
df['len'] = df['tokens'].apply(len) # tokens의 길이를 len 컬럼에 추가

In [36]:
df.head()

,code,problem_num,tokens,len
0,"M = 9\nN = 9\ndef main():\n\tfor i in range(1,...",problem001,"[M, Ġ=, Ġ9, Ċ, N, Ġ=, Ġ9, Ċ, def, Ġmain, ():, ...",83
1,"list1 = range(1,10)\nlist2 = list1\nfor i in l...",problem001,"[list, 1, Ġ=, Ġrange, (, 1, ,, 10, ), Ċ, list,...",57
2,"for x in range(1, 10):\n\tfor y in range(1, 10...",problem001,"[for, Ġx, Ġin, Ġrange, (, 1, ,, Ġ10, ):, Ċ, ĉ,...",42
3,"for i in range(1,10):\n\tfor n in range(1, 10)...",problem001,"[for, Ġi, Ġin, Ġrange, (, 1, ,, 10, ):, Ċ, ĉ, ...",43
4,"for i in range(1,10):\n\tfor k in range(1,10):...",problem001,"[for, Ġi, Ġin, Ġrange, (, 1, ,, 10, ):, Ċ, ĉ, ...",46


In [50]:
df.describe()

,len
count,45101.000000
mean,149.980688
std,122.564425
min,5.000000
25%,61.000000
50%,108.000000
75%,200.000000
max,512.000000


In [41]:
# train과 validation data set 분리
train_df, valid_df, train_label, valid_label = train_test_split(
        df,
        df['problem_num'],
        random_state=42,
        test_size=0.1,
        stratify=df['problem_num']
    )

train_df = train_df.reset_index(drop=True) # Reindexing
valid_df = valid_df.reset_index(drop=True)

In [64]:
train_df

,code,problem_num,tokens,len
0,"h,w,n = [int(input()) for i in range(3)]\nprin...",problem236,"[h, ,, w, ,, n, Ġ=, Ġ[, int, (, input, ()), Ġf...",34
1,"join,rate=map(int,input().split())\nif join>=1...",problem211,"[join, ,, rate, =, map, (, int, ,, input, ().,...",41
2,"n, k = map(int, input().split())\nh = sorted(l...",problem227,"[n, ,, Ġk, Ġ=, Ġmap, (, int, ,, Ġinput, ()., s...",46
3,"N = int(input())\na_list = list(map(int, input...",problem257,"[N, Ġ=, Ġint, (, input, ()), Ċ, a, _, list, Ġ=...",87
4,N=int(input())\nD=[{} for _ in range(N+1)]\nfo...,problem262,"[N, =, int, (, input, ()), Ċ, D, =[, {, }, Ġfo...",218
...,...,...,...,...
40585,"import math\nx1, y1, x2, y2 = map(float, input...",problem029,"[import, Ġmath, Ċ, x, 1, ,, Ġy, 1, ,, Ġx, 2, ,...",57
40586,"N, M = map(int, input().split())\ndata = [0]*N...",problem240,"[N, ,, ĠM, Ġ=, Ġmap, (, int, ,, Ġinput, ()., s...",131
40587,"mod = int(1e9+7)\ndef add(a, b):\n\tc = a + b\...",problem235,"[ow, (, a, ,, Ġb, /, 2, );, Ċ, ĉ, ĉ, return, Ġ...",512
40588,"import sys\nINF = -1\ndef main(T1, T2, A1, A2,...",problem269,"[import, Ġsys, Ċ, IN, F, Ġ=, Ġ-, 1, Ċ, def, Ġm...",245


**stratify (classification을 다룰 때 매우 중요한 옵션)**
- default=None
- stratify 값을 target으로 지정해주면 각각의 class 비율을 train / validation에 유지해 줌
    - 한 쪽에 쏠려서 분배되는 것을 방지
- 만약 이 옵션을 지정해 주지 않고 분류 문제를 다룬다면, 성능의 차이가 많이 날 수 있음

----------------
#### Create Easy Level dataset
- Random하게 뽑은 Positive pairs와 Negative pairs로 구성
**Train data set**

In [48]:
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [59]:
total_positive_pairs = []
total_negative_pairs = []

In [60]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = train_df[train_df['problem_num'] == problem]['code'].to_list()
    other_codes = train_df[train_df['problem_num'] != problem]['code'].to_list()
    
    # positive_pairs 500개 (총 300 * 500 = 150,000개) 추출
    # negative_pairs 500개 (총 300 * 500 = 150,000개) 추출
    positive_pairs = list(combinations(solution_codes,2))
    random.shuffle(positive_pairs)
    positive_pairs = positive_pairs[:500]
    random.shuffle(other_codes)
    other_codes = other_codes[:500]
    
    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))
    
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:16<00:00, 18.44it/s]


In [61]:
print("total_positive length is ", len(total_positive_pairs))
print("total_negative length is ", len(total_negative_pairs))

total_positive length is  150000
total_negative length is  150000


In [62]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
easy_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
easy_data = easy_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
easy_data.to_csv('data/train_data_Easy.csv',index=False)

**Validation data set**

In [65]:
codes = valid_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = valid_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [66]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = valid_df[valid_df['problem_num'] == problem]['code'].to_list()
    other_codes = valid_df[valid_df['problem_num'] != problem]['code'].to_list()
    
    # positive_pairs 500개 (총 300 * 50 = 15,000개) 추출
    # negative_pairs 500개 (총 300 * 50 = 15,000개) 추출
    positive_pairs = list(combinations(solution_codes,2))
    random.shuffle(positive_pairs)
    positive_pairs = positive_pairs[:50]
    random.shuffle(other_codes)
    other_codes = other_codes[:50]
    
    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))
    
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 230.86it/s]


In [67]:
print("total_positive length is ", len(total_positive_pairs))
print("total_negative length is ", len(total_negative_pairs))

total_positive length is  15000
total_negative length is  15000


In [68]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
easy_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
easy_data = easy_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
easy_data.to_csv('data/valid_data_Easy.csv',index=False)

-------------
#### Create Middle Level dataset
- 유사도가 중간인 코드들의 Positive pairs와 Negative pairs로 이루어짐
**Training data set**

In [107]:
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

# # code를 토큰화하여 저장, train_df에 저장된 모든 코드들에 대한 token들을 리스트 하나에 저장함
# tokenized_corpus = train_df['tokens'].to_list()
# # 토큰화된 code에 대해 상관관계를 계산, 현재 무작위로 설정된 code에 대해서 수행하기 때문에
# # 상관관계를 계산하는 코드는 같은 문제를 푸는 코드가 아닐 수 있음.
# bm25 = BM25Okapi(tokenized_corpus)

In [115]:
total_positive_pairs = []
total_negative_pairs = []

In [116]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_solutions로 저장
    solutions = train_df[train_df['problem_num'] == problem]
    other_solutions = train_df[train_df['problem_num'] != problem]
    
    positive_pairs = []
    negative_pairs = []
    
    # 같은 문제를 푸는 코드에 대한 토큰을 저장
    pp_tokens = solutions['tokens'].to_list()
    np_tokens = other_solutions['tokens'].to_list()
    ppbm25 = BM25Okapi(pp_tokens)
    npbm25 = BM25Okapi(np_tokens)
    
    # 각각의 코드에 대해 유사도 비교하여 pairs에 추가
    for solution, token in list(zip(solutions['code'], solutions['tokens']))[:10]: # solution_codes: 약 135
        pp_scores = ppbm25.get_scores(token)
        np_scores = npbm25.get_scores(token)
        pos_idx = round(len(pp_scores)/2) # pos_idx: 135 / 2 = 약 68
        neg_idx = round(len(np_scores)/2) # neg_idx: 40450 / 2 = 20,225
        positive_code_ranking = pp_scores.argsort()[::1][pos_idx:]
        negative_code_ranking = np_scores.argsort()[::-1][neg_idx:]
        
        # positive, negative pairs의 길이는 코드당 15개 (총 (300 * 10 * 15) * 2 = 90,000개)
        for i in range(15):
            score_idx = positive_code_ranking[i]
            positive_pairs.append((solution, train_df['code'].iloc[score_idx]))
        
        for i in range(15):
            score_idx = negative_code_ranking[i]
            negative_pairs.append((solution, train_df['code'].iloc[score_idx]))

            
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|██████████████████████████████████████████████████████████████████████████████| 300/300 [1:51:03<00:00, 22.21s/it]


In [136]:
print("total_positive length is ", len(total_positive_pairs))
print("total_negative length is ", len(total_negative_pairs))

total_positive length is  45000
total_negative length is  45000


In [135]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
middle_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
middle_data = middle_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
middle_data.to_csv('data/train_data_Middle.csv',index=False)

**validation data**

In [137]:
codes = valid_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = valid_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [138]:
total_positive_pairs = []
total_negative_pairs = []

In [139]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_solutions로 저장
    solutions = valid_df[valid_df['problem_num'] == problem]
    other_solutions = valid_df[valid_df['problem_num'] != problem]
    
    positive_pairs = []
    negative_pairs = []
    
    # 같은 문제를 푸는 코드에 대한 토큰을 저장
    pp_tokens = solutions['tokens'].to_list()
    np_tokens = other_solutions['tokens'].to_list()
    ppbm25 = BM25Okapi(pp_tokens)
    npbm25 = BM25Okapi(np_tokens)
    
    # 각각의 코드에 대해 유사도 비교하여 pairs에 추가
    for solution, token in list(zip(solutions['code'], solutions['tokens']))[:5]: # solution_codes 5개
        pp_scores = ppbm25.get_scores(token)
        np_scores = npbm25.get_scores(token)
        pos_idx = round(len(pp_scores)/2) # pos_idx: 135 / 2 = 약 68
        neg_idx = round(len(np_scores)/2) # neg_idx: 40450 / 2 = 20,225
        positive_code_ranking = pp_scores.argsort()[::1][pos_idx:]
        negative_code_ranking = np_scores.argsort()[::-1][neg_idx:]
        
        # positive, negative pairs의 길이는 코드당 3개 (총 (300 * 5 * 3) * 2 = 9,000개)
        for i in range(3):
            score_idx = positive_code_ranking[i]
            positive_pairs.append((solution, valid_df['code'].iloc[score_idx]))
        
        for i in range(3):
            score_idx = negative_code_ranking[i]
            negative_pairs.append((solution, valid_df['code'].iloc[score_idx]))

            
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [05:32<00:00,  1.11s/it]


In [140]:
print("total_positive length is ", len(total_positive_pairs))
print("total_negative length is ", len(total_negative_pairs))

total_positive length is  4500
total_negative length is  4500


In [141]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
middle_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
middle_data = middle_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
middle_data.to_csv('data/valid_data_Middle.csv',index=False)

-------------
#### Create High Level dataset
- 유사도가 낮은 Positive pairs
- 유사도가 높은 Negative pairs로 이루어짐
**Training data set**

In [142]:
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [143]:
total_positive_pairs = []
total_negative_pairs = []

In [144]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_solutions로 저장
    solutions = train_df[train_df['problem_num'] == problem]
    other_solutions = train_df[train_df['problem_num'] != problem]
    
    positive_pairs = []
    negative_pairs = []
    
    # 같은 문제를 푸는 코드에 대한 토큰을 저장
    pp_tokens = solutions['tokens'].to_list()
    np_tokens = other_solutions['tokens'].to_list()
    ppbm25 = BM25Okapi(pp_tokens)
    npbm25 = BM25Okapi(np_tokens)
    
    # 각각의 코드에 대해 유사도 비교하여 pairs에 추가
    for solution, token in list(zip(solutions['code'], solutions['tokens']))[:10]: # solution_codes: 약 135
        pp_scores = ppbm25.get_scores(token)
        np_scores = npbm25.get_scores(token)
        positive_code_ranking = pp_scores.argsort()[::1]
        negative_code_ranking = np_scores.argsort()[::-1]
        
        # positive, negative pairs의 길이는 코드당 15개 (총 (300 * 10 * 15) * 2 = 90,000개)
        for i in range(15):
            score_idx = positive_code_ranking[i]
            positive_pairs.append((solution, train_df['code'].iloc[score_idx]))
        
        for i in range(15):
            score_idx = negative_code_ranking[i]
            negative_pairs.append((solution, train_df['code'].iloc[score_idx]))

            
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|██████████████████████████████████████████████████████████████████████████████| 300/300 [2:03:02<00:00, 24.61s/it]


In [145]:
print("total_positive length is ", len(total_positive_pairs))
print("total_negative length is ", len(total_negative_pairs))

total_positive length is  45000
total_negative length is  45000


In [146]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
high_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
high_data = high_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
high_data.to_csv('data/train_data_High.csv',index=False)

**validation data**

In [147]:
codes = valid_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = valid_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [148]:
total_positive_pairs = []
total_negative_pairs = []

In [149]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_solutions로 저장
    solutions = valid_df[valid_df['problem_num'] == problem]
    other_solutions = valid_df[valid_df['problem_num'] != problem]
    
    positive_pairs = []
    negative_pairs = []
    
    # 같은 문제를 푸는 코드에 대한 토큰을 저장
    pp_tokens = solutions['tokens'].to_list()
    np_tokens = other_solutions['tokens'].to_list()
    ppbm25 = BM25Okapi(pp_tokens)
    npbm25 = BM25Okapi(np_tokens)
    
    # 각각의 코드에 대해 유사도 비교하여 pairs에 추가
    for solution, token in list(zip(solutions['code'], solutions['tokens']))[:5]: # solution_codes 5개
        pp_scores = ppbm25.get_scores(token)
        np_scores = npbm25.get_scores(token)
        positive_code_ranking = pp_scores.argsort()[::1]
        negative_code_ranking = np_scores.argsort()[::-1]
        
        # positive, negative pairs의 길이는 코드당 3개 (총 (300 * 5 * 3) * 2 = 9,000개)
        for i in range(3):
            score_idx = positive_code_ranking[i]
            positive_pairs.append((solution, valid_df['code'].iloc[score_idx]))
        
        for i in range(3):
            score_idx = negative_code_ranking[i]
            negative_pairs.append((solution, valid_df['code'].iloc[score_idx]))

            
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [07:27<00:00,  1.49s/it]


In [150]:
print("total_positive length is ", len(total_positive_pairs))
print("total_negative length is ", len(total_negative_pairs))

total_positive length is  4500
total_negative length is  4500


In [151]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
high_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
high_data = high_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
high_data.to_csv('data/valid_data_High.csv',index=False)